In [15]:
import trino
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [10, 8]

conn = trino.dbapi.connect(
    host='kakaoent-presto-adhoc.kakaoent.io',
    port=8443,
    user='samuel-ch',
    catalog='hadoop_kent',
    schema='default',                          # 사용할 Database 지정
    http_scheme='https',
    auth=trino.auth.BasicAuthentication("samuel-ch", "melon!Q2w3e"),  # 카카오 LDAP이 아닌 하둡 LDAP 입력
    http_headers={'X-Presto-Time-Zone': 'Asia/Seoul'},
)
cur = conn.cursor()

In [9]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
  ]

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '11px')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]

cm = sns.light_palette("green", as_cmap=True)

In [16]:
tmp_query = '''
SELECT DT
     , EVENT_CNT
     , IF(EVENT_CNT > 0,'Y','N') AS EVENT_YN
     , NEW_PROD_CNT
     , NEW_JOIN_PROD_CNT
     , SUSPND_RELEASE_PROD_CNT
FROM(     
SELECT DT
     , COUNT(DISTINCT EVENT_ID) AS EVENT_CNT
     , COUNT(DISTINCT BUY_NO) AS NEW_PROD_CNT
     , COUNT(DISTINCT IF(PROD_FLUC_PRT_CD = 100, BUY_NO)) AS NEW_JOIN_PROD_CNT
     , COUNT(DISTINCT IF(PROD_FLUC_PRT_CD = 101, BUY_NO)) AS SUSPND_RELEASE_PROD_CNT
FROM(
    SELECT T1.DT
         , T1.FIRST_BUY_NO
         , T1.BUY_NO
         , T2.PROD_PRT_CD
         , T1.USER_MKEY AS MEMBER_KEY
         , T3.PROD_FLUC_PRT_CD -- 100 신규 101 일시정지해제
    FROM HADOOP_KENT.MELON_MA_STAT_PRODUCTION.F_PROD_USER_FXMT_DT  T1
    JOIN HADOOP_KENT.MELON_MA_STAT_PRODUCTION.D_PROD T2 ON (T1.PROD_ID = T2.PROD_ID)
    JOIN HADOOP_KENT.MELON_MA_STAT_PRODUCTION.D_PROD_JOIN_PRT T3 ON (T1.PROD_JOIN_PRT_CD = T3.PROD_JOIN_PRT_CD)
    WHERE T1.LOG_DATE BETWEEN '20240101' AND '20241231'
      AND PF_YN = 1
      AND T2.PROD_ATTR_CD NOT IN (10040, 10050)
      AND T1.PROD_JOIN_PRT_CD IN (1100, 1200, 1201, 1205, 1206, 1203, 1207)
      AND T2.PROD_SELL_PRT_CD IN (60000) /*B2C*/
    GROUP BY 1,2,3,4,5,6  
) F1
LEFT JOIN(
    SELECT EVENT_ID
         , TITLE
         , DATE(START_DATE) AS START_DATE
         , DATE(END_DATE) AS END_DATE
    FROM HADOOP_KENT.MELON_ODS_POC_PRODUCTION.EVENT_INFO_RO
    WHERE DATE(START_DATE) >= DATE('2024-01-01') 
    AND END_DATE <= DATE('2024-12-31')
    AND CATEGORY IN ('100068','100067')
) F2 ON F1.DT BETWEEN F2.START_DATE AND F2.END_DATE
GROUP BY 1
)
ORDER BY 1
'''

In [18]:
cur.execute(tmp_query)

HttpError: error 400: b'User must be set'

In [ ]:


results = cur.fetchall()
colnames = [part[0] for part in cur.description]

tmp_df = pd.DataFrame(results, columns=colnames)

cur.cancel()
conn.close()
tmp_df.head()

HttpError: error 400: b'User must be set'

In [ ]:
tmp_df.head()

In [8]:
(1256869 - 890448)/890448*100

41.150185075377784

In [11]:
341/31034

0.010987948701424244

In [15]:
3200*16

51200